### Scrapes the data using selenium and stores it in json files

In [ ]:
import pandas as pd
import json
import os


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service
from selenium.common.exceptions import NoSuchElementException
import time

# Load the CSV file
file_path = "updated_instagram.csv"
df = pd.read_csv(file_path)

# Path to the Edge WebDriver executable
edge_driver_path = "C:\\Users\\Harsh Dipak Marke\\Data Science Project\\msedgedriver.exe"

# Initialize Microsoft Edge WebDriver with the specified path
service = Service(edge_driver_path)
driver = webdriver.Edge(service=service)

# URL of the Instagram login page
login_url = 'https://www.instagram.com/accounts/login/'

# Load the Instagram login page
driver.get(login_url)

# Wait for the page to load
time.sleep(5)

# Find the username and password input fields and enter your credentials
username_input = driver.find_element(By.NAME, "username")
password_input = driver.find_element(By.NAME, "password")

# Replace 'your_username' and 'your_password' with your actual Instagram credentials
username_input.send_keys("ds_harsh_1")
password_input.send_keys("Harsh@123")

# Submit the login form
password_input.send_keys(Keys.RETURN)

# Wait for the login process to complete
time.sleep(5)


for index, row in df.iloc[160:].iterrows():
    url = row['Instagram_url']
    username = row['Instagram name']
    
    # URL of the Instagram profile you want to scrape
    profile_url = url

    # Load the Instagram profile page
    driver.get(profile_url)

    # Wait for the page to load
    time.sleep(5)

    try:
        followers_element = driver.find_element(By.XPATH, f'//a[contains(@href, "/{username}/followers/")]/span')
        followers_count = followers_element.text
    except NoSuchElementException:
        followers_count = "0"

    try:
        following_element = driver.find_element(By.XPATH, f'//a[contains(@href, "/{username}/following/")]/span')
        following_count = following_element.text
    except NoSuchElementException:
        following_count = "0"

    try:
        posts_element = driver.find_element(By.XPATH, '//span[@class="html-span xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x1hl2dhg x16tdsg8 x1vvkbs"]')
        posts_count = posts_element.text
    except NoSuchElementException:
        posts_count = "0"

    # Example: Scraping the URL of the profile picture
    profile_picture_elements = driver.find_elements(By.XPATH, '//img[contains(@alt, "\'s profile picture")]')
    if profile_picture_elements:
        profile_picture_url = profile_picture_elements[0].get_attribute("src")
    else:
        profile_picture_url = "#"

    # Find all the div tags with the specified class name
    post_divs = driver.find_elements(By.CLASS_NAME, '_aabd')

    # Initialize an empty list to store information about each post
    posts_info = []

    # Iterate through each div tag to extract post information
    
    for post_div in post_divs:
        post_info = {}
        # Extract post URL
        post_a_tag = post_div.find_element(By.TAG_NAME, 'a')
        post_info['url'] = post_a_tag.get_attribute('href')

        # Extract post image URL
        post_img_tag = post_div.find_element(By.TAG_NAME, 'img')
        post_info['urlImage'] = post_img_tag.get_attribute('src')

        # Extract post description (alt text)
        description = post_img_tag.get_attribute('alt')
        post_info['description'] = description if description else ""

        # Extract mentions (strings starting with '@') from the description
        mentions = [word for word in description.split() if word.startswith('@')]
        post_info['mentions'] = mentions

        posts_info.append(post_info)

    likes_counts = []

    # Iterate through each URL in the array
    if len(posts_info) == 0:
        driver.quit()
        
    post_count = 0
    zero_likes = 0
    for post in posts_info:
        # Load the URL
        driver.get(post['url'])
        time.sleep(5)  
        post_count = post_count + 1

        try:
            # Find the span tag containing the likes count
            likes_span = driver.find_element(By.XPATH, '//span[@class="html-span xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x1hl2dhg x16tdsg8 x1vvkbs"]')

            # Extract the text from the span tag
            likes_count = likes_span.text
        except NoSuchElementException:
            likes_count = "0"  # Set likes count to "0" if element is not found
            zero_likes = zero_likes + 1

        # Append the likes count to the list
        likes_counts.append(likes_count)

    # Close the WebDriver
    if(post_count != 0 and post_count == zero_likes):
        driver.quit()

    # Append the likes count to the corresponding post
    i = 0
    for post in posts_info:
        post['likes'] = likes_counts[i]
        i += 1

    user_data = {}
    # Define the path to the folder
    folder_path = "JSON-files"

    # Create the folder if it doesn't exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    user_data['username'] = username
    user_data['profile_url'] = profile_url
    user_data['followers'] = followers_count
    user_data['following'] = following_count
    user_data['number_of_posts'] = posts_count
    user_data['posts_info'] = posts_info

    json_file_path = os.path.join(folder_path, f"{username}.json")

    # Write the user data to the JSON file
    with open(json_file_path, "w") as json_file:
        json.dump(user_data, json_file, indent=4)

    print(f"JSON file saved successfully at: {json_file_path}")

    
driver.quit()